In [1]:
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import re
import numpy as np

In [2]:
qual_eval_folder = './evaluation'
items_path = "./data/games.pkl"
data_path = "./data/interactions_splits_0"
reviews_path = "./data/reviews.parquet"

From the base recommender results, we will now continue to use the ideal combination of methods as a baseline before we make further improvements.
This means we use cosine distance and normalized feature vectors. We also don't use feedback weighting.

The following cell runs the basic recommender and provides more in-depth evaluation and compares it to a popularity-based recommender.

In [4]:
rec = PopBasedRecommender(train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet")
rec.generate_recommendations() # read_max=1000
rec.evaluate(k=10)

{'HR@10': 0.6325869461648403,
 'nDCG@10': 0.1987056756228976,
 'recall@10': 0.21966528783489012,
 'ideal_recall@10': 0.9186804883480776,
 'nRecall@10': 0.23557168190445943}

In [11]:
dim_red = TruncatedSVD(n_components=100, random_state=500)
use_data = ['specs', 'genres', 'tags', 'early_access', 'publisher']
rec = ImprovedRecommender(items_path, train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf='smooth', use_feedback=True, normalize=True, dim_red=None, columns=use_data)
rec.generate_recommendations() # read_max=1000
rec.evaluate(k=10)

62970it [01:43, 607.99it/s]
62970it [02:09, 484.52it/s] 


{'HR@10': 0.3581070350960775,
 'nDCG@10': 0.08424816307810852,
 'recall@10': 0.09530128094021754,
 'ideal_recall@10': 0.9186804883480776,
 'nRecall@10': 0.10370435279082253}

In [10]:
dim_red = TruncatedSVD(n_components=100, random_state=500)
use_data = ['specs', 'genres', 'tags', 'early_access', 'publisher']
rec = ImprovedRecommender(items_path, train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf='smooth', use_feedback=True, normalize=True, dim_red=None, columns=use_data)
rec.generate_recommendations(read_max=1000) # read_max=1000
rec.evaluate(k=10)

1000it [00:05, 190.95it/s]
1000it [00:04, 202.79it/s]


{'HR@10': 0.699,
 'nDCG@10': 0.1423581917601125,
 'recall@10': 0.024164494301903396,
 'ideal_recall@10': 0.18434941324984938,
 'nRecall@10': 0.1313}

In [5]:
items = pd.read_pickle(items_path)
items["price"] = items["price"].apply(lambda p: np.float32(p) if re.match(r"\d+(?:.\d{2})?", str(p)) else 0)
items["metascore"] = items["metascore"].apply(lambda m: m if m != "NA" else np.nan)
items["developer"].fillna(value='', inplace=True)
items["developer"] = items["developer"].apply(lambda my_str: my_str.lower().split(','))
items["publisher"].fillna(value='', inplace=True)
items["publisher"] = items["publisher"].apply(lambda my_str: my_str.lower().split(','))
items["early_access"] = items["early_access"].apply(lambda x: ["earlyaccess"] if x else [])
items["specs"] = items["specs"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])
items["tags"] = items["tags"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])
items["genres"] = items["genres"].apply(lambda l: [re.subn(r"[^a-z0-9]", "", my_str.lower())[0] for my_str in l])

items

,publisher,genres,app_name,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore,users_count
0,[valve],[action],Counter-Strike: Global Offensive,2012-08-21,"[fps, multiplayer, shooter, action, teambased,...",NaN,"[multiplayer, steamachievements, fullcontrolle...",14.99,[],730,[valve],Very Positive,83.0,42620
1,[valve],"[indie, simulation]",Garry's Mod,2006-11-29,"[sandbox, multiplayer, funny, moddable, buildi...",NaN,"[singleplayer, multiplayer, coop, crossplatfor...",9.99,[],4000,[facepunch studios],Overwhelmingly Positive,NaN,42157
2,[smartly dressed games],"[action, adventure, casual, freetoplay, indie]",Unturned,2017-07-07,"[freetoplay, survival, zombies, multiplayer, o...",NaN,"[singleplayer, onlinemultiplayer, onlinecoop, ...",0.00,[],304930,[smartly dressed games],Very Positive,NaN,37654
3,[valve],[action],Left 4 Dead 2,2009-11-16,"[zombies, coop, fps, multiplayer, action, onli...",NaN,"[singleplayer, multiplayer, coop, steamachieve...",19.99,[],550,[valve],Overwhelmingly Positive,89.0,35990
4,[re-logic],"[action, adventure, indie, rpg]",Terraria,2011-05-16,"[sandbox, adventure, survival, 2d, multiplayer...",NaN,"[singleplayer, multiplayer, onlinemultiplayer,...",9.99,[],105600,[re-logic],Overwhelmingly Positive,83.0,28551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,[projectile entertainment],[indie],Momentum,2016-08-11,"[indie, physics]",NaN,"[singleplayer, steamachievements, fullcontroll...",9.99,[],462130,[projectile entertainment],8 user reviews,NaN,1
7346,[nuligine],"[action, indie, racing, simulation]",MadOut Ice Storm,2015-09-04,"[racing, indie, action, simulation]",NaN,"[singleplayer, fullcontrollersupport]",6.99,[],400500,[nuligine],Mixed,NaN,1
7347,[like a boss llc],"[action, adventure, indie, strategy]",OCCHIO,2016-08-11,"[action, adventure, indie, strategy, puzzlepla...",NaN,"[singleplayer, fullcontrollersupport]",1.99,[],513140,[acerio games],7 user reviews,NaN,1
7348,[dxf games],"[action, indie, rpg]",Just a Cleric,2016-05-19,"[rpg, indie, action, pixelgraphics, difficult]",NaN,"[singleplayer, steamachievements, steamtrading...",4.99,[],467330,[dxf games],Positive,NaN,1


In [7]:
reviews = pd.read_parquet(reviews_path)
reviews

# user_ids = pd.read_parquet('./data/user_ids.parquet')
# user_ids

,reviews,recommend
index,,
0,"[192, 594, 1560, 3118, 3546]","[False, False, False, False, True]"
3,"[65, 93, 139, 266, 476, 1684]","[True, True, True, True, True, True, True, Tru..."
5,"[15, 20, 353, 451, 3123, 3843]","[True, True, True, True, True, True, True]"
6,"[0, 7, 883, 1906]","[False, True, True, True]"
7,[64],[True]
...,...,...
62732,[19],[True]
62857,[1],[True]
62883,[5],[True]
